In [2]:
# Libraries
import json, pandas as pd, pydeck as pdk

# Exloratory analysis

The project aim is to create a Machine Learning model capable of detecting the dates when a crop field has been manured, using satellite data. <br>
Before even starting considering models, it is useful to perform exploratory analysis on the obtained crop fields dataset.

## Import the JSON file containing all crop fields details

In [3]:
# Load JSON data from file
with open("../Datasets/crop-fields.json") as f:
    data = json.load(f)

# Create DataFrame with properties
fields_df = pd.DataFrame([f["properties"] for f in data["features"]])

# Add column with coordinates for each field
fields_df["polygon_coordinates"] = [[tuple(c) for c in p] for f in data["features"] for p in f["geometry"]["coordinates"]]

In [4]:
fields_df

,crop_field_name,manure_dates,polygon_coordinates
0,P-BLD,[2022-05-26],"[(-4.202723286616649, 43.39683579015289), (-4...."
1,P-BLLT1,[2022-05-16],"[(-4.085622203603083, 43.429605845026266), (-4..."
2,P-BLLT2,[2022-05-26],"[(-4.084840437376829, 43.430826294936246), (-4..."
3,P-Cardana,[2022-02-24],"[(8.658803437240303, 45.85842753378426), (8.65..."
4,P-CBRCS1,[2022-05-26],"[(-4.200826431306206, 43.39067464298489), (-4...."
5,P-CBRCS2,[2022-05-26],"[(-4.204911872695676, 43.3876170244562), (-4.2..."
6,P-CLGT,[2022-05-16],"[(-4.111699726693341, 43.39830644556494), (-4...."
7,P-CLMBRS,[2022-05-26],"[(-4.544769098140127, 43.38040395682432), (-4...."
8,P-CMNTR,[2022-05-16],"[(-4.147208715069137, 43.40038457218137), (-4...."
9,P-DR,[2022-03-21],"[(-4.142486752802821, 43.396858931472195), (-4..."


## Show crop fields on a map

In [9]:
# Define the layer with a tooltip
layer = pdk.Layer(
    "PolygonLayer",
    data=fields_df,
    get_polygon="polygon_coordinates",
    get_fill_color=[255, 255, 0, 100],
    get_line_color=[255, 255, 0, 100],
    stroked=True,
    filled=True,
    lineWidthMinPixels=3,
    pickable=True,
    auto_highlight=True,
)

# Define the initial view state of the map
view_state = pdk.ViewState(
    latitude=43.38,
    longitude=-4.80,
    zoom=8
)

# Create the map with the layers and the initial view state
r = pdk.Deck(layers=layer, initial_view_state=view_state,)

# Show the map
r


{
  "initialViewState": {
    "latitude": 43.38,
    "longitude": -4.8,
    "zoom": 8
  },
  "layers": [
    {
      "@@type": "PolygonLayer",
      "autoHighlight": true,
      "data": [
        {
          "crop_field_name": "P-BLD",
          "manure_dates": [
            "2022-05-26"
          ],
          "polygon_coordinates": [
            [
              -4.202723286616649,
              43.39683579015289
            ],
            [
              -4.20272409742545,
              43.396789898748956
            ],
            [
              -4.202417062727523,
              43.39626953971194
            ],
            [
              -4.202164935080123,
              43.3962851312709
            ],
            [
              -4.20174114605577,
              43.39635334429408
            ],
            [
              -4.2012127508798365,
              43.396544340350516
            ],
            [
              -4.201763255273048,
              43.397284469281985
            ],
            [
              -4.201771301900092,
              43.39730395840358
            ],
            [
              -4.202723286616649,
              43.39683579015289
            ]
          ]
        },
        {
          "crop_field_name": "P-BLLT1",
          "manure_dates": [
            "2022-05-16"
          ],
          "polygon_coordinates": [
            [
              -4.085622203603083,
              43.429605845026266
            ],
            [
              -4.08556587721377,
              43.42963311523658
            ],
            [
              -4.085496139779383,
              43.42960974077134
            ],
            [
              -4.085292291894251,
              43.42943443199443
            ],
            [
              -4.085120630517298,
              43.429188998853675
            ],
            [
              -4.085158181443506,
              43.429165624216935
            ],
            [
              -4.085107219472223,
              43.429056542459485
            ],
            [
              -4.083232680688746,
              43.430143118756874
            ],
            [
              -4.083605507741816,
              43.43047425325417
            ],
            [
              -4.083938101659663,
              43.43078201192734
            ],
            [
              -4.0840185679301095,
              43.4309105688844
            ],
            [
              -4.084577935859985,
              43.43042820143843
            ],
            [
              -4.086020964309998,
              43.42963737073982
            ],
            [
              -4.085622203603083,
              43.429605845026266
            ]
          ]
        },
        {
          "crop_field_name": "P-BLLT2",
          "manure_dates": [
            "2022-05-26"
          ],
          "polygon_coordinates": [
            [
              -4.084840437376829,
              43.430826294936246
            ],
            [
              -4.084454199278684,
              43.430575023915516
            ],
            [
              -4.084025045836301,
              43.43093342567431
            ],
            [
              -4.083944579565855,
              43.43121196470595
            ],
            [
              -4.0839177574821806,
              43.431286955720715
            ],
            [
              -4.083933850729795,
              43.4313580510982
            ],
            [
              -4.083982130450276,
              43.43140674648393
            ],
            [
              -4.08405186792645,
              43.431443754950884
            ],
            [
              -4.084038456881376,
              43.431441807137475
            ],
            [
              -4.084207436049314,
              43.43132493821355
            ],
            [
              -4.084349593127103,
              43.43121196470595
            ],
            [
              -4.084600380445416,
              43.431018156137

It can be noticed that our fields are placed in the Northern part of Spain. Please consider generalization issue.